In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-05 17:31:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-03-05 17:31:17 (6.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



Load RDS Data into Spark DataFrame

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

BEAUTY Vine Table

In [4]:
#Configure settings for RDS
jdbc_url="jdbc:postgresql://postgres.cxyspgwfkz9n.us-east-2.rds.amazonaws.com:5432/postgres2" # copied Endpoint & /database name goes here
config = {"user":"root", 
          "password": "class2021", 
          "driver":"org.postgresql.Driver"}

In [80]:
# Read vine table in RDS to vine_df SparkDataframe
beauty_df1 = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
beauty_df1.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3I2DHQBR577SS|          5|            0|          0|   N|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|
| RAI2OIG50KZ43|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



Beauty Vine Data Summary and Descriptive Statistics: 


In [81]:
beauty_df2 = beauty_df1.describe("star_rating","helpful_votes", "total_votes")
beauty_df2.show()

+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|           5115453|           5115452|           5115452|
|   mean| 4.187214150382019| 1.708924646346012| 2.163590040528188|
| stddev|1.2997552151973897|17.249077699006353|18.433556383382943|
|    min|                 1|                 0|                 0|
|    max|                 5|             13362|             13636|
+-------+------------------+------------------+------------------+




PAID Reviews Analysis Helpfulness: 

In [95]:
paid_reviewers = beauty_df1.filter(beauty_df1['vine'] == 'Y')
paid_reviewers.count()

33309

In [96]:
paid_stats = paid_reviewers.describe("star_rating","helpful_votes", "total_votes")
paid_stats.show()

+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|             33309|             33309|             33309|
|   mean| 4.021435648023057| 2.374253204839533|3.0267195052388245|
| stddev|0.9821680026595679|23.471693511283544| 25.44707856479592|
|    min|                 1|                 0|                 0|
|    max|                 5|              1778|              1827|
+-------+------------------+------------------+------------------+



In [97]:
# Finding the Percentage for helpful_reviews that are based on 'Y' vine Paid reviews 
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import round
paid_reviewers = beauty_df1.filter(beauty_df1['vine'] == 'Y')
df_percent = paid_reviewers.withColumn('paid_percent',f.col('helpful_votes')/f.col('total_votes'))
df_percent_filtered = df_percent.filter(df_percent['paid_percent'] > .05)
df_percent_filtered.sort(df_percent_filtered["helpful_votes"], ascending=False).show()

+--------------+-----------+-------------+-----------+----+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|      paid_percent|
+--------------+-----------+-------------+-----------+----+------------------+
|R2GVZP2VX03H24|          4|         1778|       1827|   Y|0.9731800766283525|
| R54SQFPUX7ZXZ|          3|         1338|       1362|   Y|0.9823788546255506|
|R36R4BR12VTR30|          5|          994|       1196|   Y|0.8311036789297659|
| RSQ5772HO77WB|          5|          986|       1021|   Y|0.9657198824681684|
|R3PUA3BURP7I1K|          5|          950|        959|   Y|0.9906152241918665|
| RGCJVKPPXLQ8K|          4|          863|        953|   Y|0.9055613850996852|
|R106V6GUNWRSSS|          5|          857|        889|   Y|0.9640044994375703|
|R23S154O5MEJTO|          2|          852|        917|   Y|0.9291166848418757|
|R11PCOTB05TIIM|          4|          790|        986|   Y|0.8012170385395537|
|R1YUECR5IDRSIT|          4|          735|        76

NOT Paid Reviews Analysis Helpfulness: 

In [92]:
unpaid_reviewers = beauty_df1.filter(beauty_df1['vine'] == 'N')
unpaid_reviewers.count()

5082143

In [93]:
unpaid_stats = unpaid_reviewers.describe("star_rating","helpful_votes", "total_votes")
unpaid_stats.show()

+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|           5082143|           5082143|           5082143|
|   mean|4.1883006833928125|1.7045639998717077|2.1579329822084894|
| stddev| 1.301511560930214| 17.20078839542214|18.378632432224393|
|    min|                 1|                 0|                 0|
|    max|                 5|             13362|             13636|
+-------+------------------+------------------+------------------+



In [94]:
# Finding the Percentage for helpful_reviews that are based on 'N' vine unpaid reviews 
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import round
unpaid_reviewers = beauty_df1.filter(beauty_df1['vine'] == 'N')
df_percent = unpaid_reviewers.withColumn('unpaid_percent',f.col('helpful_votes')/f.col('total_votes'))
df_percent_filtered = df_percent.filter(df_percent['unpaid_percent'] > .05)
df_percent_filtered.sort(df_percent_filtered["helpful_votes"], ascending=False).show()

+--------------+-----------+-------------+-----------+----+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|    unpaid_percent|
+--------------+-----------+-------------+-----------+----+------------------+
|R2WXS4OG6MTBUN|          1|        13362|      13636|   N|0.9799061308301554|
| R7U56MY458IO9|          4|        10265|      10374|   N|0.9894929631771737|
|R2H7SCLBX4FGXJ|          4|         6245|       6382|   N| 0.978533375117518|
|R2LI72CVT761C9|          5|         5789|       5895|   N|0.9820186598812553|
|R2B9YUBVVDZQ7P|          5|         5693|       5844|   N|0.9741615331964408|
|R3AEMZ1NMNAGA2|          5|         4786|       4888|   N|0.9791325695581015|
| R1JQ4IMFLNR40|          5|         3994|       4365|   N|0.9150057273768614|
| R58XKR9WDXV5X|          1|         3916|       4068|   N|0.9626352015732547|
| R2AW92C3VGO9U|          1|         3864|       4266|   N|0.9057665260196905|
| RYGFW7WFPOH8G|          1|         3809|       400

In [130]:
paid_reviewers = beauty_df1.filter(beauty_df1['vine'] == 'Y')
Paid_FiveStar_Votes = paid_reviewers.filter(beauty_df1['star_rating'] == 5)
Paid_FiveStar_Votes.count()

12429

In [134]:
# Finding the Percentage of 5-Star reviews are based on 'Y' vine paid reviews 
# What % of Y vine = 5star 
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import round
Paid_FiveStar_Votes = paid_reviewers.filter(beauty_df1['star_rating'] == 5)
paid_percent = Paid_FiveStar_Votes.withColumn('paid_percent',f.col('helpful_votes')/f.col('total_votes'))
paid_percent_filtered = paid_percent.filter(paid_percent['paid_percent'] > .05)
paid_percent_filtered.sort(paid_percent_filtered["helpful_votes"], ascending=False).show()

+--------------+-----------+-------------+-----------+----+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|      paid_percent|
+--------------+-----------+-------------+-----------+----+------------------+
|R36R4BR12VTR30|          5|          994|       1196|   Y|0.8311036789297659|
| RSQ5772HO77WB|          5|          986|       1021|   Y|0.9657198824681684|
|R3PUA3BURP7I1K|          5|          950|        959|   Y|0.9906152241918665|
|R106V6GUNWRSSS|          5|          857|        889|   Y|0.9640044994375703|
|R1EK0ZMP0ZSYME|          5|          470|        514|   Y| 0.914396887159533|
| RMYE4J5331ET3|          5|          462|        487|   Y|0.9486652977412731|
|R2F4KBBD3MTMKP|          5|          390|        423|   Y|0.9219858156028369|
| RHWGXN7QI3RZV|          5|          313|        322|   Y|0.9720496894409938|
|R1Y4OY1LZ29JO7|          5|          299|        308|   Y|0.9707792207792207|
|R11IYWLYRH2M02|          5|          292|        31

In [109]:
unpaid_reviewers = beauty_df1.filter(beauty_df1['vine'] == 'N')
Unpaid_FiveStar_Votes = unpaid_reviewers.filter(beauty_df1['star_rating'] == 5)
Unpaid_FiveStar_Votes.count()

3242324

In [110]:
# Finding the Percentage of 5-Star reviews are based on 'N' vine paid reviews 
# What % of N vine = 5star 
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import round
Unpaid_FiveStar_Votes = unpaid_reviewers.filter(beauty_df1['star_rating'] == 5)
Unpaid_percent = Unpaid_FiveStar_Votes.withColumn('unpaid_percent',f.col('helpful_votes')/f.col('total_votes'))
Unpaid_percent_filtered = Unpaid_percent.filter(Unpaid_percent['unpaid_percent'] > .05)
Unpaid_percent_filtered.sort(Unpaid_percent_filtered["helpful_votes"], ascending=False).show()

+--------------+-----------+-------------+-----------+----+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|    unpaid_percent|
+--------------+-----------+-------------+-----------+----+------------------+
|R2LI72CVT761C9|          5|         5789|       5895|   N|0.9820186598812553|
|R2B9YUBVVDZQ7P|          5|         5693|       5844|   N|0.9741615331964408|
|R3AEMZ1NMNAGA2|          5|         4786|       4888|   N|0.9791325695581015|
| R1JQ4IMFLNR40|          5|         3994|       4365|   N|0.9150057273768614|
|R3UM7PK8WT0DH1|          5|         3681|       3789|   N|0.9714964370546318|
|R3F9G1IIEZW2Z5|          5|         3622|       3722|   N|0.9731327243417518|
|R3KH8BMPJS0OVP|          5|         3354|       3504|   N|0.9571917808219178|
|R3N1RNUHEADN19|          5|         3301|       3644|   N|0.9058726673984633|
|R2QE1D2IUXBESL|          5|         3096|       3129|   N|0.9894534995206136|
| RIDVWLE45YW7I|          5|         3091|       317

BOOKS Vine Table

In [111]:
#Configure settings for RDS
jdbc_url="jdbc:postgresql://postgres.cxyspgwfkz9n.us-east-2.rds.amazonaws.com:5432/postgres" # copied Endpoint & /database name goes here
config = {"user":"root", 
          "password": "class2021", 
          "driver":"org.postgresql.Driver"}

In [112]:
# Read vine table in RDS to vine_df SparkDataframe
books_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
books_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



Books Vine Data Summary and Descriptive Statistics: 

In [113]:
books_stats = books_df.describe("star_rating","helpful_votes", "total_votes")
books_stats.show()

+-------+------------------+----------------+------------------+
|summary|       star_rating|   helpful_votes|       total_votes|
+-------+------------------+----------------+------------------+
|  count|           3105515|         3105515|           3105515|
|   mean| 4.182722672406992|9.33469038146652|12.400078891906817|
| stddev|1.2458976835774067|26.2239470287692| 30.24433583508727|
|    min|                 1|               0|                 0|
|    max|                 5|           15835|             16352|
+-------+------------------+----------------+------------------+



In [124]:
from pyspark.sql.functions import round
FiveStar_count = books_df[books_df['star_rating']== 5].count()
FiveStar_vote = books_df.groupBy("helpful_votes").count() 
FiveStar_vote  = FiveStar_vote.orderBy(FiveStar_vote["count"].desc())
FiveStar_vote .withColumn('percentage', round(FiveStar_vote["count"]/FiveStar_count,3)).show()

+-------------+------+----------+
|helpful_votes| count|percentage|
+-------------+------+----------+
|            0|459923|     0.247|
|            1|408855|     0.219|
|            2|333718|     0.179|
|            3|269067|     0.144|
|            4|215443|     0.116|
|            5|176301|     0.095|
|            6|144403|     0.077|
|            7|120318|     0.065|
|            8|101095|     0.054|
|            9| 85924|     0.046|
|           10| 73994|      0.04|
|           11| 64172|     0.034|
|           12| 56530|      0.03|
|           13| 50035|     0.027|
|           14| 43759|     0.023|
|           15| 38900|     0.021|
|           16| 34403|     0.018|
|           17| 30954|     0.017|
|           18| 27856|     0.015|
|           19| 24957|     0.013|
+-------------+------+----------+
only showing top 20 rows



In [ ]:
beauty_vine_df = beauty_vine_df.filter(beauty_vine_df['total_votes'] >= 20)
beauty_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2QRUE9REK8OUC|          5|           18|         23|   N|
|R2POXRW6PXHXZG|          5|           25|         30|   N|
| RZN43XRZ89IIJ|          3|           10|         27|   N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|
| R46UL5G5HEPRZ|          5|           18|         20|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [128]:
paid_reviewers = books_df.filter(books_df['vine'] == 'Y')
paid_stats = paid_reviewers.describe("star_rating","helpful_votes", "total_votes")
paid_stats.show()

+-------+------------------+-------------+------------------+
|summary|       star_rating|helpful_votes|       total_votes|
+-------+------------------+-------------+------------------+
|  count|                 2|            2|                 2|
|   mean|               4.5|          3.0|               4.5|
| stddev|0.7071067811865476|          0.0|2.1213203435596424|
|    min|                 4|            3|                 3|
|    max|                 5|            3|                 6|
+-------+------------------+-------------+------------------+



In [129]:
unpaid_reviewers = books_df.filter(books_df['vine'] == 'N')
unpaid_stats = unpaid_reviewers.describe("star_rating","helpful_votes", "total_votes")
unpaid_stats.show()

+-------+------------------+-----------------+------------------+
|summary|       star_rating|    helpful_votes|       total_votes|
+-------+------------------+-----------------+------------------+
|  count|           3105513|          3105513|           3105513|
|   mean|4.1827224680753226|9.334694461108358|12.400083979683872|
| stddev| 1.245897994135481| 26.2239549803491|30.244344885565138|
|    min|                 1|                0|                 0|
|    max|                 5|            15835|             16352|
+-------+------------------+-----------------+------------------+

